# Basic validation notebook. Duplicate for your use.

In [ ]:
# Importing tval is from the parent directory of notebooks. The parent is appended
# to PYTHONPATH
import os, sys
sys.path.append("..")

In [1]:
from pathlib import Path
import pathlib
import geopandas as gpd
from tval import trace_validator
import time

/home/nialov/.local/share/virtualenvs/trace-validator-GOA3psTM/lib/python3.8/site-packages/geopandas/_compat.py:84: UserWarning: The Shapely GEOS version (3.8.0-CAPI-1.13.1 ) is incompatible with the GEOS version PyGEOS was compiled with (3.8.1-CAPI-1.13.3). Conversions between both will be slow.
  warnings.warn(


ModuleNotFoundError: No module named 'tval'

In [ ]:
# Setup snap thresholds here
snap_threshold = 0.001
snap_threshold_multiplier = 1.1
area_edge_snap_multiplier = 5

In [ ]:
trace_validator.BaseValidator.set_snap_threshold_and_multipliers(snap_threshold, snap_threshold_multiplier, area_edge_snap_multiplier)

In [ ]:
# Add trace and area filepaths to traces and areas list
traces = [
    Path(""),
               ]
areas = [
    Path(""), 
]

# All files should exist
assert all([p.exists() for p in traces+areas])

In [ ]:
# For use after validation if debugging or further analysis is wanted
gdfs = []

print(f"Validating with snap threshold: {snap_threshold} and multiplier: {snap_threshold_multiplier}")
for trace, area in zip(traces, areas):

    print(f"Validating {trace}")
    trace_gdf = gpd.read_file(trace, mode="r")
    # [print(t) if t.is_empty or t is None else "" for t in trace_gdf.geometry]
    if trace_validator.BaseValidator.ERROR_COLUMN in trace_gdf.columns:
        print(f"Error column already in gdf. Old will be backed up ({trace})")
        trace_gdf[f"error_column_backup"] = trace_gdf[trace_validator.BaseValidator.ERROR_COLUMN]
        trace_gdf[trace_validator.BaseValidator.ERROR_COLUMN] = [
            [] for _ in trace_gdf.index
        ]
    area_gdf = gpd.read_file(area, mode="r")
    for validator in (
    trace_validator.GeomNullValidator
    , trace_validator.GeomTypeValidator
    , trace_validator.MultiJunctionValidator
    , trace_validator.VNodeValidator
    , trace_validator.MultipleCrosscutValidator
    , trace_validator.UnderlappingSnapValidator
    , trace_validator.TargetAreaSnapValidator
    ):
        print(f"Validating with {validator}")
        print(f"Start time is {time.asctime()}")
        # change auto_fix here. By default True. Null geometries will be removed even if auto_fix is False
        trace_gdf = validator.execute(trace_gdf, area_gdf, auto_fix=True, parallel=True)
        gdfs.append(trace_gdf.copy())
    # When saving to GPKG, validation error column is transformed to str type
    trace_gdf.astype(
        {trace_validator.BaseValidator.ERROR_COLUMN: 'str'}
    ).to_file(
        Path(f""), driver="GPKG"
    )